In [1]:
from bokeh.models import Circle
from bokeh.plotting import figure, show, output_file
from bokeh.sampledata.us_cities import data as cities
from bokeh.sampledata.us_counties import data as counties
from bokeh.sampledata.us_states import data as states
import math
import seaborn as sns

elastic = {'dgreen': '#176655',
           'lgreen': '#95c63d',
           'pink': '#e9478c',
           'purple': '#2c458f',
           'teal': '#3cbeb1',
           'yellow': '#f4bd19'}

In [2]:
df = pd.read_table('city_state_ads.tsv')

In [3]:
def rgb_to_hex(rgb):
    """helper func to deal with some seaborn-to-bokeh weirdness"""
    return '#%02x%02x%02x' % tuple(max(0, min(255, int(255*x))) for x in rgb[:3])

# Palettes for circle interiors and borders (assumes we want a color gradient)
circ_fill_pltt = [rgb_to_hex(x) for x in sns.light_palette(elastic['dgreen'], 200)][100:]
circ_border_pltt = [rgb_to_hex(x) for x in sns.light_palette(elastic['lgreen'], 200)][100:]

Create the plot and draw the background, turning gridlines and axes off.

In [4]:
p = figure(title="Ads Per City", toolbar_location="left", plot_width=1100, plot_height=700)
p.axis.visible = None
p.background_fill_color = elastic['teal']
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

Draw state backgrounds

In [5]:
# Draw state backgrounds
EXCLUDED = ('AK', 'HI')
state_xs = [states[code]["lons"] for code in states if code not in EXCLUDED]
state_ys = [states[code]["lats"] for code in states if code not in EXCLUDED]

p.patches(state_xs, state_ys,
          fill_alpha=1.0,
          fill_color='#cccccc',
          line_alpha=0.8,
          line_color='#000000',
          line_width=1)

Draw patches for counties

In [6]:
# Don't need these counties
EXCLUDED = ("ak", "hi", "pr", "gu", "vi", "mp", "as")
county_xs = [counties[code]["lons"] for code in counties if counties[code]["state"] not in EXCLUDED]
county_ys = [counties[code]["lats"] for code in counties if counties[code]["state"] not in EXCLUDED]

p.patches(county_xs, county_ys,
          fill_alpha=0.0,
          line_alpha=1.0,
          line_color='#737f76',
          line_width=0.5)

Draw circles for cities.

In [7]:
df = pd.read_table('city_state_ads.tsv')

scalar= 70.
max_ads = max(df.Ads)

circles = df.apply(lambda x:
                   Circle(x=x['Long'],
                          y=x['Lat'],
                          size=scalar*x['Ads']/max_ads+5,
                          fill_alpha=min(0.9, max(0.6, 1.*x['Ads']/max_ads)),
                          fill_color=circ_fill_pltt[min(99, int((100.*x['Ads']/max_ads)))],
                          line_alpha=0.9,
                          line_color=circ_fill_pltt[min(99, int(100.*x['Ads']/max_ads))],
                          line_width=2.0),
                   axis=1)

for c in circles:
    p.add_glyph(c)

Write the graphic to a file and render the plot.

In [8]:
output_file("ads_cities.html", title="Map of Ads per City")
show(p)